In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
recipe_paths = '../datas/recommender_data/'

movie = pd.read_csv(recipe_paths + "recommender system table.csv" , encoding = 'cp949')
# meta = pd.read_csv(movie_paths + 'movies_metadata.csv', low_memory=False)
# meta = meta.rename(columns={'id':'movieId'})
movie.head()

In [ ]:
movie['member_id'] = movie['member_id'].astype(str)
# meta['movieId'] = meta['movieId'].astype(str)

# movie = pd.merge(movie, meta[['member_id', 'CKG_NM']], on='movieId')
movie['one'] = 1
movie.head()

In [ ]:
df = movie.pivot_table(index='member_id', columns = 'CKG_NM', values = 'one').fillna(0)
df.head()

In [ ]:
watching_metrix = df.iloc[:, : ].values

In [ ]:
# 환경설정
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')
print(DEVICE)

In [ ]:
# 학습 조건 설정
BATCH_SIZE = 64
EPOCHS = 100

In [ ]:
class CustomDataset(Dataset):

  # 데이터 정의
  def __init__(self, x_data, y_data = None):
    self.x_data = x_data
    self.y_data = y_data

  # 이 데이터 셋의 총 데이터 수
  def __len__(self):
    return len(self.x_data)

  # 어떠한 idx를 받았을 때 그에 맞는 데이터를 반환
  def __getitem__(self, idx):
    if self.y_data is None:
      x = torch.FloatTensor(self.x_data[idx])
      return x
    else:
      x = torch.FloatTensor(self.x_data[idx])
      y = torch.FloatTensor(self.y_data[idx])
      return x, y

In [ ]:
# AutoEncoder 모델 설계
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.fc1_1 = nn.Linear(4005, 15)
        self.fc1_2 = nn.Linear(4005, 15)
        self.relu = nn.ReLU()
                        
    def encode(self, x):
        mu = self.relu(self.fc1_1(x))
        log_var = self.relu(self.fc1_2(x))
                
        return mu,log_var
    
    def reparametrize(self, mu, log_var):
        std = log_var.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_().to(DEVICE)
        
        return eps.mul(std).add_(mu)
    
    def forward(self,x):
        mu, log_var = self.encode(x)
        reparam = self.reparametrize(mu,log_var)
        
        return mu,log_var,reparam
        
encoder = Encoder().to(DEVICE)

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.fc1 = nn.Linear(15, 4005)
        self.simoid = nn.Sigmoid()
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.simoid(out)
        
        return out
        
decoder = Decoder().to(DEVICE)

In [ ]:
reconstruction_function = nn.MSELoss(size_average=False)

def loss_function(recon_x, x, mu, log_var):
    MSE = reconstruction_function(recon_x, x)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(log_var.exp()).mul_(-1).add_(1).add_(log_var)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return MSE + KLD

parameters = list(encoder.parameters())+ list(decoder.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.0005)

In [ ]:
def train(encoder, decoder, train_loader):
  encoder.train()
  decoder.train()
  train_loss = 0

  for feature in train_loader:

    feature = feature.to(DEVICE)
    optimizer.zero_grad()
    mu,log_var,reparam = encoder(feature)
    output = decoder(reparam)
    loss = loss_function(output, feature, mu, log_var)
    loss.backward()
    optimizer.step()
    
    train_loss += loss.item()

  train_loss /= len(train_loader)
  return train_loss

In [ ]:
def evaluate(encoder, decoder, train_loader):
  encoder.eval()
  decoder.eval()
  result = []

  with torch.no_grad():

    for feature in train_loader:
      feature = feature.to(DEVICE)
      mu,log_var,reparam = encoder(feature)
      output = decoder(reparam)
      result.append(output.cpu().numpy())

  result = np.concatenate(result)
  return result

In [ ]:
train_dataset = CustomDataset(watching_metrix)

train_loader = DataLoader(
  train_dataset,
  batch_size = BATCH_SIZE,
  shuffle = False,
  drop_last = False)

for epoch in range(1, EPOCHS + 1):
  train_loss = train(encoder, decoder, train_loader)
  print(f"\n[EPOCH: {epoch}], \tTrain Loss: {train_loss:.4f}")

In [ ]:
result = evaluate(encoder, decoder, train_loader)
result

In [ ]:
watching_metrix

In [ ]:
movie_li = df.columns.tolist()
result[watching_metrix >= 1] = -1

In [ ]:
recommend_li = []
for i in result.argmax(axis=1):
  recommend_li.append(movie_li[i])

In [ ]:
recommend_df = pd.DataFrame()
recommend_df['user_id'] = df.index.tolist()
recommend_df['recipe_name'] = recommend_li
recommend_df.head(15)

In [ ]:
recommend_df[recommend_df['user_id'] == '7']

In [ ]:
for i in df.columns.tolist():
  if '돼지고기' in i:
    print(i)

In [ ]:
df.iloc[1]

In [ ]:
for i in df.iloc[12][df.iloc[12] >= 1].index.tolist():
  if '돼지고기' in i:
    print(i)